In [3]:
import speech_recognition as sr

recognizer = sr.Recognizer()

with sr.Microphone() as source:
    print("🎤 Nói đi...")
    recognizer.adjust_for_ambient_noise(source)
    audio = recognizer.listen(source)

try:
    text = recognizer.recognize_google(audio, language="vi-VN")
    print("📝 Bạn nói:", text)
except sr.UnknownValueError:
    print("❌ Không nhận diện được")
except sr.RequestError as e:
    print("❌ Lỗi dịch vụ:", e)


🎤 Nói đi...
📝 Bạn nói: alo 1 2 3 4 5 6 9 10
